## Playing around with Tensorflow Eager Execution...

In [1]:
# do imports (use nightly builds of tensorflow)
import tensorflow as tf
import tensorflow.contrib.eager as tfe
from tensorflow.examples.tutorials.mnist import input_data

In [2]:
# enable eager execution
tfe.enable_eager_execution()

In [3]:
# try some execution
x = [[2.]]
m = tf.matmul(x, x)
print(m)

tf.Tensor([[ 4.]], shape=(1, 1), dtype=float32)


In [4]:
# A MNIST model
class MNISTModel(tfe.Network):
    def __init__(self):
        super(MNISTModel, self).__init__()
        self.layer1 = self.track_layer(tf.layers.Dense(units=10))
        self.layer2 = self.track_layer(tf.layers.Dense(units=10))
    def call(self, input):
        result = self.layer1(input)
        result = self.layer2(result)
        return result
    
def loss_function(model, x, y):
    y_ = model(x)
    return tf.nn.softmax_cross_entropy_with_logits(labels=y, logits=y_)

In [5]:
model = MNISTModel()
batch = tf.zeros([1, 1, 784])

result = model(batch)
print(result)

tf.Tensor([[[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]]], shape=(1, 1, 10), dtype=float32)


In [18]:
optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.001)
mnist = input_data.read_data_sets("/tmp/data/", one_hot=True)
dataset = tf.data.Dataset.from_tensor_slices((mnist.train.images,
                                                 mnist.train.labels))

counter = 0
for (x, y) in tfe.Iterator(dataset):
    x = tf.reshape(x, (1,-1))
    y = tf.reshape(y, (1,-1))
    grads = tfe.implicit_gradients(loss_function)(model, x, y)
    optimizer.apply_gradients(grads)
    if(counter % 5000 == 0):
        print('The loss for the %d round is %f'%(counter, loss_function(model, x, y)))
    counter += 1    

Extracting /tmp/data/train-images-idx3-ubyte.gz
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz
The loss for the 0 round is 0.346763
The loss for the 5000 round is 0.000323
The loss for the 10000 round is 0.093344
The loss for the 15000 round is 0.000720
The loss for the 20000 round is 0.233394
The loss for the 25000 round is 0.003513
The loss for the 30000 round is 0.002840
The loss for the 35000 round is 0.000868
The loss for the 40000 round is 0.057147
The loss for the 45000 round is 0.139772
The loss for the 50000 round is 0.026651


In [27]:
def compute_accuracy(predictions, labels):
    return tf.reduce_sum(
    tf.cast(
        tf.equal(
            tf.argmax(predictions, axis=1,
                    output_type=tf.int64),
            tf.argmax(labels, axis=1,
                    output_type=tf.int64)),
        dtype=tf.float32)) / float(predictions.shape[0].value)